### Number Theoretic Transform - CRT


O primeiro passo é a escolha de um $N$ da forma $2^d$  e um primo $\,q\,$ que  verifique $\,q \equiv 1 \bmod 2N\,$.

o corpo $\,\mathbb{F}_q\,$ contém todas as raízes do polinómio $\,\phi \equiv w^N + 1$.  Seja $\xi$ uma qualquer destas raízes; então todas as raízes têm a forma $\,\xi^{2\,i+1}$ , com $\;i=0,\cdots,N-1$.


In [1]:
class NTT(object):
#    
    def __init__(self, n=128, q=None):
        if not  n in [32,64,128,256,512,1024,2048]:
            raise ValueError("improper argument ",n)
        self.n = n  
        if not q:
            self.q = 1 + 2*n
            while True:
                if (self.q).is_prime():
                    break
                self.q += 2*n
        else:
            if q % (2*n) != 1:
                raise ValueError("Valor de 'q' não verifica a condição NTT")
            self.q = q
             
        self.F = GF(self.q) ;  self.R = PolynomialRing(self.F, name="w")
        w = (self.R).gen()
        
        g = (w^n + 1)
        xi = g.roots(multiplicities=False)[-1]
        self.xi = xi
        rs = [xi^(2*i+1)  for i in range(n)] 
        self.base = crt_basis([(w - r) for r in rs])  
    
    
    def ntt(self,f):
        def _expand_(f): 
            u = f.list()
            return u + [0]*(self.n-len(u)) 
        
        def _ntt_(xi,N,f):
            if N==1:
                return f
            N_ = N/2 ; xi2 =  xi^2  
            f0 = [f[2*i]   for i in range(N_)] ; f1 = [f[2*i+1] for i in range(N_)] 
            ff0 = _ntt_(xi2,N_,f0) ; ff1 = _ntt_(xi2,N_,f1)  
    
            s  = xi ; ff = [self.F(0) for i in range(N)] 
            for i in range(N_):
                a = ff0[i] ; b = s*ff1[i]  
                ff[i] = a + b ; ff[i + N_] = a - b 
                s = s * xi2                     
            return ff 
        
        return _ntt_(self.xi,self.n,_expand_(f))
        
    def ntt_inv(self,ff):                              ## transformada inversa
        return sum([ff[i]*self.base[i] for i in range(self.n)])
    
    def random_pol(self,args=None):
        return (self.R).random_element(args)

In [2]:
# Teste

#T = NTT(n=1024)
T = NTT(n=2048,q=343576577)

f = T.random_pol(1023)
# print(f)

ff = T.ntt(f)

fff = T.ntt_inv(ff)

# print(fff)
print("Correto ? ",f == fff)

NameError: name 'GF' is not defined